In [1]:
import os
import sys
import random
from pathlib import Path

from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
import statsmodels.api as sm
from statsmodels.tsa.vector_ar.var_model import VAR

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import xgboost as xgb

SEED = 2103
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def eval_fn(y_pred, y_target):
    return mean_absolute_error(y_pred, y_target)

# Load Train/Test Data

In [2]:
train_df = pd.read_pickle('./data_postprocessing/train_df.pkl')
test_df = pd.read_pickle('./data_postprocessing/test_df.pkl')

In [3]:
train_df.head(2)

,window_id,ask_price_0,ask_size_0,bid_price_0,bid_size_0,far_price_0,imbalance_buy_sell_flag_0,imbalance_size_0,matched_size_0,near_price_0,...,bid_size_198,far_price_198,imbalance_buy_sell_flag_198,imbalance_size_198,matched_size_198,near_price_198,reference_price_198,target_198,wap_198,last_5_mins
0,0,1.000026,8493.03,0.999812,60651.5,0.0,1,3180602.69,13380276.64,0.0,...,54300.05,0.0,1,15249373.90,26134518.94,0.0,0.999775,-0.510216,1.000000,0
1,0,1.000026,23519.16,0.999812,13996.5,0.0,1,1299772.70,15261106.63,0.0,...,153691.34,0.0,1,13496480.93,27604966.30,0.0,1.000288,-1.419783,1.000222,0


In [4]:
test_df.head(2)

,window_id,ask_price_0,ask_size_0,bid_price_0,bid_size_0,far_price_0,imbalance_buy_sell_flag_0,imbalance_size_0,matched_size_0,near_price_0,...,bid_size_198,far_price_198,imbalance_buy_sell_flag_198,imbalance_size_198,matched_size_198,near_price_198,reference_price_198,target_198,wap_198,last_5_mins
0,433,1.000066,2765.73,0.999697,12685.14,0.0,-1,5128680.68,10543243.05,0.0,...,5753.28,0.0,-1,26806621.39,83364970.70,0.0,1.000486,5.459785,1.000000,0
1,433,0.999882,78944.32,0.999697,2331.03,0.0,-1,3901807.90,11733675.67,0.0,...,77927.44,0.0,-1,26782684.93,83388907.16,0.0,0.999944,3.089905,1.000051,0


In [5]:
feature_cols = [col for col in train_df if not col.startswith("target") and not col.startswith("window_id")]
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(train_df[feature_cols].values)

MinMaxScaler(feature_range=(-1, 1))

In [6]:
class StockDataset(Dataset):
    """
    To preserve original window structure, do not shuffle, and set batch size to be 55.
    """
    def __init__ (self, df):
        self.target_cols = [col for col in df if col.startswith("target")] # we have to change this when we add lags
        self.feature_cols = [col for col in df if not col.startswith("target") and not col.startswith("window_id")]
        self.labels = df[self.target_cols] # labels for all stocks at all times
        self.features = df[self.feature_cols]
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return self.features.iloc[idx].values, self.labels.iloc[idx].values

In [7]:
train_dataset = torch.load('./data_postprocessing/train.pt')
test_dataset = torch.load('./data_postprocessing/test.pt')
train_dataloader = DataLoader(train_dataset, batch_size=55, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=55, shuffle=False)

In [8]:
x, y = train_dataset[0]
print(x.shape, y.shape)

(2036,) (185,)


# XGBoost

In [ ]:
temp_train_loader = DataLoader(train_dataset, batch_size=len(train_dataset))
X, y = next(iter(temp_train_loader))
X, y = X.numpy(), y.numpy()

In [ ]:
print(X.shape, y.shape)

(23815, 2036) (23815, 185)


In [ ]:
temp_test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))
test_X, test_y = next(iter(temp_test_loader))
test_X, test_y = test_X.numpy(), test_y.numpy()

In [ ]:
print(test_X.shape, test_y.shape)

(2640, 2036) (2640, 185)


In [ ]:
xgb_model = xgb.XGBRegressor(device="cuda")
xgb_model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
train_y_hat = xgb_model.predict(X)
print(f'Train MAE: {eval_fn(train_y_hat, y)})')

/home/mingchonglim/anaconda3/envs/mlip/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [22:02:03] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Train MAE: 2.33017348998886)


In [ ]:
test_y_hat = xgb_model.predict(test_X)
print(f'Test MAE: {eval_fn(test_y_hat, test_y)})')

Test MAE: 6.913688415179943)


# VARMAX

In this part, we do a rolling prediction using VARMAX as follows (the `|` operator is a concatenation along the 0th axis, and each `train/test[i]` refers to a window of 55 samples): \
`(WAP | Target)_train1 | .. | (WAP | Target)_trainN | WAP_test1 -> Pred_Target_test1` \
`(WAP | Target)_train1 | .. | (WAP | Target)_trainN | (WAP | Pred_Target)_test1 | WAP_test2 -> Pred_Target_test2` \
and so on... \
\
We then just use the last 100 pairs of (WAP | Target) from this big array, plus the current WAP_test appended at the end, to predict the current Target_test \
\
This may perform poorly, but serves as a good baseline for other models to beat

In [ ]:
wap_cols = [col for col in train_df if col.startswith("wap")]
target_cols = [col for col in train_df if col.startswith("target")]
exo_cols = [col for col in train_df if not col.startswith("wap") and not col.startswith("target")]

num_train_samples = len(train_df)
wap_indices = [i * 55 + j for i in range(0, (num_train_samples * 2) // 55, 2) for j in range(55)]
target_indices = [55 + i * 55 + j for i in range(0, (num_train_samples * 2) // 55, 2) for j in range(55)]
concat_train_arr = np.empty((num_train_samples * 2, len(target_cols)))
concat_train_arr[wap_indices, :] = train_df[wap_cols].values
concat_train_arr[target_indices, :] = train_df[target_cols].values

# exogenous variables are just duplicated
exo_train_arr = np.empty((num_train_samples * 2, len(exo_cols)))
exo_train_arr[wap_indices, :] = train_df[exo_cols].values
exo_train_arr[target_indices, :] = train_df[exo_cols].values

In [ ]:
num_test_samples = len(test_df)

curr_test_wap_and_target_arr = test_df[wap_cols][:55]
curr_test_exo_arr = test_df[exo_cols][:55]

total_MAE = 0.0

for i in tqdm(range(num_test_samples // 55)):
    # train inputs
    curr_concat_train_arr = np.concatenate([concat_train_arr, curr_test_wap_and_target_arr])
    curr_exo_train_arr = np.concatenate([exo_train_arr, curr_test_exo_arr])

    # scale inputs
    curr_scaler = MinMaxScaler()
    curr_scaler.fit(curr_concat_train_arr)
    curr_concat_train_arr = curr_scaler.transform(curr_concat_train_arr)

    # Fit the VAR model
    model = VAR(endog=curr_concat_train_arr[-55*100-55*101:])
    model_fit = model.fit()

    # Make prediction on validation
    curr_pred = model_fit.forecast(model.endog, steps=55)

    # evaluate
    curr_true_target = test_df[target_cols][i*55:(i+1)*55].values
    curr_MAE = mean_absolute_error(curr_pred, curr_true_target)
    total_MAE += curr_MAE

    # concat new inputs
    if i != num_test_samples // 55 - 1:
        curr_test_wap_and_target_arr = np.concatenate([
            curr_test_wap_and_target_arr,
            curr_pred,
            test_df[wap_cols][(i+1)*55:(i+2)*55]
        ])
        curr_test_exo_arr = np.concatenate([curr_test_exo_arr, test_df[exo_cols][(i+1)*55:(i+2)*55], test_df[exo_cols][(i+1)*55:(i+2)*55]])

    print(curr_MAE)

print(f'Test MAE:', total_MAE / (num_test_samples // 55))

  2%|▏         | 1/48 [00:00<00:12,  3.67it/s]

5.626262182701441


  4%|▍         | 2/48 [00:00<00:12,  3.75it/s]

6.199536437138377


  6%|▋         | 3/48 [00:00<00:12,  3.71it/s]

...

5.117107175166288


 98%|█████████▊| 47/48 [00:13<00:00,  3.39it/s]

5.592916167068363


100%|██████████| 48/48 [00:13<00:00,  3.58it/s]

4.798367229914929
Test MAE: 5.8855244395116415


# LSTM
In this part we train a simple bi-directional LSTM

In [ ]:
class LSTM(nn.Module):
    def __init__(self, hidden=256, out_dim=185):
        super().__init__()
        self.hidden = hidden
        self.out_dim = out_dim
        self.lstm = nn.LSTM(2036, hidden_size=self.hidden, num_layers=2, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(self.hidden * 2, self.out_dim)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [ ]:
lstm_model = LSTM().to(device)
lstm_optimizer = torch.optim.Adam(lstm_model.parameters())
loss_fn = nn.L1Loss()

for epoch in range(30):
    print(f"--- Epoch [{epoch + 1}/{30}] ---")
    total_epoch_loss = 0.
    for step, batch in enumerate(train_dataloader):
        features, targets = batch
        features = torch.from_numpy(scaler.transform(features))
        features, targets = features.float(), targets.float()
        features, targets = features.to(device), targets.to(device)
        targets = targets.unsqueeze(0)
        features = features.unsqueeze(0)

        preds = lstm_model(features)
        loss = loss_fn(targets, preds)
        total_epoch_loss += loss.detach().cpu()

        if step % 100 == 0:
            print("Loss:", loss.item())

        lstm_optimizer.zero_grad()
        loss.backward()
        lstm_optimizer.step()
    print(f"Epoch Loss: {total_epoch_loss / len(train_dataloader)}")

--- Epoch [1/30] ---
Loss: 5.549416542053223
Loss: 5.237961292266846
Loss: 8.095149040222168
Loss: 6.798412799835205
Loss: 5.570059776306152
Epoch Loss: 6.318953037261963
--- Epoch [2/30] ---
Loss: 5.544959545135498
Loss: 5.238502025604248
Loss: 8.09648609161377
Loss: 6.800079345703125
Loss: 5.575272560119629
Epoch Loss: 6.317306041717529
--- Epoch [3/30] ---
Loss: 5.541306495666504
Loss: 5.239346504211426
Loss: 8.089181900024414
Loss: 6.8014750480651855
Loss: 5.572729587554932
Epoch Loss: 6.3166728019714355
--- Epoch [4/30] ---
Loss: 5.542489528656006
Loss: 5.236428737640381
Loss: 8.09494400024414
Loss: 6.800201416015625
Loss: 5.568076133728027
Epoch Loss: 6.315825462341309
--- Epoch [5/30] ---
Loss: 5.5440497398376465
Loss: 5.2335333824157715
Loss: 8.132625579833984
Loss: 6.76423978805542
Loss: 5.565145492553711
Epoch Loss: 6.310324668884277
--- Epoch [6/30] ---
Loss: 5.535722732543945
Loss: 5.228562355041504
Loss: 8.079418182373047
Loss: 6.7209882736206055
Loss: 5.56702184677124
Epo

In [ ]:
lstm_model.eval()
total_loss = 0.
total_elems = 0
for step, batch in enumerate(test_dataloader):
    features, targets = batch
    features = torch.from_numpy(scaler.transform(features))
    features, targets = features.float(), targets.float()
    features, targets = features.to(device), targets.to(device)
    targets = targets.unsqueeze(0)
    features = features.unsqueeze(0)
    with torch.inference_mode():
        preds = lstm_model(features)
        total_loss += torch.sum(torch.abs(preds - targets)).item()
        total_elems += torch.numel(targets)
print(f"MAE: {total_loss / total_elems}")

MAE: 5.760513931011978


# Conditional DDPM
In this part we treat the inputs and outputs as 1-channel images and apply conditional DDPM

In [8]:
from diffusion_model import Unet, p_losses, sample
U = Unet(dim=64, channels=1, dim_mults=(1, 2, 4), self_condition=True)
if torch.cuda.is_available():
    U.cuda()
    print('Models moved to GPU.')
u_optimizer = torch.optim.Adam(U.parameters(), 0.0002, [0.5, 0.999])

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Models moved to GPU.


In [ ]:
num_epc = 5
for epoch in range(num_epc):
    print(f"--- Epoch [{epoch+1}/{num_epc}] ---")

    total_epoch_loss = 0.

    for step, batch in enumerate(train_dataloader):
        cond, targets = batch
        cond = torch.from_numpy(scaler.transform(cond))
        cond, targets = cond.float(), targets.float()
        cond, targets = cond.to(device), targets.to(device)
        targets = targets.unsqueeze(0).unsqueeze(0)
        cond = cond.unsqueeze(0)
        targets = torch.nn.functional.pad(targets, (0, 71, 0, 9), "constant", 0)

        # 1. Sample t uniformally for every example in the batch
        t = torch.randint(low=0, high=500, size=(1,), device=device).long()

        # 2. Get l1 loss
        loss = p_losses(U, targets, t, loss_type='l1', time_cond=cond)

        if step % 100 == 0:
            print("Loss:", loss.item())
    
        total_epoch_loss += loss.detach().item()

        u_optimizer.zero_grad()
        loss.backward()
        u_optimizer.step()
    
    print("Epoch Loss:", total_epoch_loss / len(train_dataloader))

    # Save every epoch
    print("Saving...")
    torch.save(U.state_dict(), "diffusion.pth")

--- Epoch [1/5] ---
Loss: 0.8664166331291199
Loss: 0.342109739780426
Loss: 0.5751746892929077
Loss: 0.36596542596817017
Loss: 0.6054816246032715
Epoch Loss: 0.5488351859479248
Saving...
--- Epoch [2/5] ---
Loss: 0.48535293340682983
Loss: 0.5322597026824951
Loss: 0.36875081062316895
Loss: 0.5387202501296997
Loss: 0.27861088514328003
Epoch Loss: 0.4736612992528955
Saving...
--- Epoch [3/5] ---
Loss: 0.2649272382259369
Loss: 0.243866428732872
Loss: 0.5191173553466797
Loss: 0.293118953704834
Loss: 0.5198401212692261
Epoch Loss: 0.4562738733946864
Saving...
--- Epoch [4/5] ---
Loss: 0.7090773582458496
Loss: 0.5887994766235352
Loss: 0.47108373045921326
Loss: 0.5159890055656433
Loss: 0.32491758465766907
Epoch Loss: 0.44573564989186876
Saving...
--- Epoch [5/5] ---
Loss: 0.4543801248073578
Loss: 0.4828573763370514
Loss: 0.42861083149909973
Loss: 0.48692965507507324
Loss: 0.3912566900253296
Epoch Loss: 0.44426095021101397
Saving...


In [ ]:
# sampling loop
U = Unet(dim=64, channels=1, dim_mults=(1, 2, 4), self_condition=True)
if torch.cuda.is_available():
    U.cuda()
U.load_state_dict(torch.load('diffusion.pth'))

U.eval()
total_loss = 0.
total_elems = 0
for step, batch in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
    cond, targets = batch
    cond = torch.from_numpy(scaler.transform(cond))
    cond, targets = cond.float(), targets.float()
    cond, targets = cond.to(device), targets.to(device)
    targets = targets.unsqueeze(0).unsqueeze(0)
    cond = cond.unsqueeze(0)
    targets = torch.nn.functional.pad(targets, (0, 71, 0, 9), "constant", 0)
    # torch inference_mode already annotated for sample() function
    samples = sample(U, (64, 256), batch_size=1, channels=1, time_cond=cond)
    total_loss += torch.sum(torch.abs(torch.from_numpy(samples[-1]) - targets.detach().cpu())).item()
    total_elems += torch.numel(targets)
    print(total_loss / total_elems)
print(f"MAE: {total_loss / total_elems}")

  0%|          | 0/48 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

  2%|▏         | 1/48 [01:11<56:12, 71.76s/it]

4.201510429382324


sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

...

 98%|█████████▊| 47/48 [56:21<01:11, 71.94s/it]

4.384790430677698


sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 48/48 [57:33<00:00, 71.96s/it]

4.376183301210403
MAE: 4.376183301210403


# TimeGrad

In this part we implement TimeGrad based on the original paper, and apply to our time series, trying to transform the WAP from our data into the target

In [8]:
CONTEXT_LENGTH = 55
HIDDEN_SIZE = 256
NUM_LAYERS = 2
NUM_CELLS = 256
RESIDUAL_LAYERS = 8
RESIDUAL_CHANNELS = 256
DILATION_CYCLE_LENGTH = 2
RESIDUAL_HIDDEN = 256
CONDITIONING_LENGTH = HIDDEN_SIZE

In [9]:
x, y = train_dataset[0]
num_features, num_target_features = x.shape[0], y.shape[0]
feature_cols = [col for col in train_df if not col.startswith("target") and not col.startswith("window_id")]
wap_idx = [i for i, col in enumerate(feature_cols) if col.startswith("wap")]
assert len(wap_idx) == num_target_features

time_grad_model = TimeGrad(
    num_features=num_features,
    num_target_features=num_target_features,
    context_length=CONTEXT_LENGTH,
    non_covariate_col_idx=wap_idx,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    num_cells=NUM_CELLS,
    residual_layers=RESIDUAL_LAYERS,
    residual_channels=RESIDUAL_CHANNELS,
    dilation_cycle_length=DILATION_CYCLE_LENGTH,
    residual_hidden=RESIDUAL_HIDDEN,
    conditioning_length=CONDITIONING_LENGTH
).to(device)

In [10]:
time_grad_optimizer = torch.optim.Adam(time_grad_model.parameters())

for epoch in range(30):
    print(f"--- Epoch [{epoch + 1}/{30}] ---")
    total_epoch_loss = 0.
    for step, batch in enumerate(train_dataloader):
        features, targets = batch
        features, targets = features.float(), targets.float()
        features, targets = features.to(device), targets.to(device)
        features, targets = features.unsqueeze(0), targets.unsqueeze(0)

        loss, _, _ = time_grad_model(features, targets)
        total_epoch_loss += loss.detach().cpu()

        if step % 100 == 0:
            print("Loss:", loss.item())

        time_grad_optimizer.zero_grad()
        loss.backward()
        time_grad_optimizer.step()
    print(f"Epoch Loss: {total_epoch_loss / len(train_dataloader)}")
    
    # Save every epoch
    print("Saving...")
    torch.save(time_grad_model.state_dict(), "timegrad.pth")

--- Epoch [1/30] ---
Loss: 0.9918442964553833
Loss: 0.764417827129364
Loss: 0.9035812616348267
Loss: 0.8236040472984314
Loss: 0.8140358328819275
Epoch Loss: 0.8518611788749695
Saving...
--- Epoch [2/30] ---
Loss: 0.8248192667961121
Loss: 0.8697466254234314
Loss: 0.8730795383453369
Loss: 0.90981525182724
Loss: 0.859169602394104
Epoch Loss: 0.8389438986778259
Saving...
--- Epoch [3/30] ---
Loss: 0.8506718277931213
Loss: 0.8634642958641052
Loss: 0.9100229740142822
Loss: 0.9128495454788208
Loss: 0.8314695954322815
Epoch Loss: 0.8396666646003723
Saving...
--- Epoch [4/30] ---
Loss: 0.8246649503707886
Loss: 0.8713688254356384
Loss: 0.9252825379371643
Loss: 0.9046648740768433
Loss: 0.7982223033905029
Epoch Loss: 0.8388842344284058
Saving...
--- Epoch [5/30] ---
Loss: 0.78267902135849
Loss: 0.8183034062385559
Loss: 0.8853309154510498
Loss: 0.8482010364532471
Loss: 0.7952440977096558
Epoch Loss: 0.8380935788154602
Saving...
--- Epoch [6/30] ---
Loss: 0.8352630138397217
Loss: 0.804884672164917
L

In [12]:
x, y = train_dataset[0]
num_features, num_target_features = x.shape[0], y.shape[0]
feature_cols = [col for col in train_df if not col.startswith("target") and not col.startswith("window_id")]
wap_idx = [i for i, col in enumerate(feature_cols) if col.startswith("wap")]
assert len(wap_idx) == num_target_features

time_grad_model = TimeGradPredictionNetwork(
    num_parallel_samples=1,
    prediction_length=CONTEXT_LENGTH,
    num_features=num_features,
    num_target_features=num_target_features,
    context_length=CONTEXT_LENGTH,
    non_covariate_col_idx=wap_idx,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    num_cells=NUM_CELLS,
    residual_layers=RESIDUAL_LAYERS,
    residual_channels=RESIDUAL_CHANNELS,
    dilation_cycle_length=DILATION_CYCLE_LENGTH,
    residual_hidden=RESIDUAL_HIDDEN,
    conditioning_length=CONDITIONING_LENGTH
).to(device)

In [13]:
# sampling loop
time_grad_model.load_state_dict(torch.load('timegrad.pth'))

total_loss = 0.
total_elems = 0
for step, batch in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
    features, targets = batch
    features, targets = features.float(), targets.float()
    features, targets = features.to(device), targets.to(device)
    features, targets = features.unsqueeze(0), targets.unsqueeze(0)

    with torch.inference_mode():
        samples = time_grad_model(features)
    total_loss += torch.sum(torch.abs(samples[0] - targets)).item()
    total_elems += torch.numel(targets)
    print(total_loss / total_elems)
print(f"MAE: {total_loss / total_elems}")

  2%|▏         | 1/48 [00:00<00:44,  1.05it/s]

7.694186885749386


  4%|▍         | 2/48 [00:01<00:43,  1.07it/s]

7.911542920761671


  6%|▋         | 3/48 [00:02<00:41,  1.07it/s]

...

8.00874849775665


 98%|█████████▊| 47/48 [00:44<00:00,  1.06it/s]

8.007391068534686


100%|██████████| 48/48 [00:45<00:00,  1.07it/s]

7.995133343570844
MAE: 7.995133343570844


# TimeGrad-Adapted Conditional Diffusion

In this part we do our own version of TimeGrad, where we run the input time series through an LSTM and also condition on its hidden states to do our diffusion.

In [8]:
from cond_diffusion_model import Unet, p_losses, sample
U = Unet(dim=128, channels=1, dim_mults=(1, 2, 4, 8), self_condition=True)
if torch.cuda.is_available():
    U.cuda()
    print('Models moved to GPU.')
u_optimizer = torch.optim.Adam(U.parameters(), 0.0001, [0.5, 0.999])

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Models moved to GPU.


In [9]:
num_epc = 5
for epoch in range(num_epc):
    print(f"--- Epoch [{epoch+1}/{num_epc}] ---")

    total_epoch_loss = 0.

    for step, batch in enumerate(train_dataloader):
        cond, targets = batch
        cond = torch.from_numpy(scaler.transform(cond))
        cond, targets = cond.float(), targets.float()
        cond, targets = cond.to(device), targets.to(device)
        targets = targets.unsqueeze(0).unsqueeze(0)
        cond = cond.unsqueeze(0)
        targets = torch.nn.functional.pad(targets, (0, 71, 0, 9), "constant", 0)

        # 1. Sample t uniformally for every example in the batch
        t = torch.randint(low=0, high=500, size=(1,), device=device).long()

        # 2. Get l1 loss
        loss = p_losses(U, targets, t, loss_type='l2', time_cond=cond)

        if step % 100 == 0:
            print("Loss:", loss.item())
    
        total_epoch_loss += loss.detach().item()

        u_optimizer.zero_grad()
        loss.backward()
        u_optimizer.step()
    
    print("Epoch Loss:", total_epoch_loss / len(train_dataloader))

    # Save every epoch
    print("Saving...")
    torch.save(U.state_dict(), "cond_diffusion.pth")

--- Epoch [1/5] ---
Loss: 1.354584813117981
Loss: 0.2350679486989975
Loss: 0.6375114917755127
Loss: 0.28499627113342285
Loss: 0.7179057002067566
Epoch Loss: 0.6123815074253303
Saving...
--- Epoch [2/5] ---
Loss: 0.4805217683315277
Loss: 0.6244086027145386
Loss: 0.2914869785308838
Loss: 0.6305547952651978
Loss: 0.17532482743263245
Epoch Loss: 0.48986321815029304
Saving...
--- Epoch [3/5] ---
Loss: 0.16021402180194855
Loss: 0.12643244862556458
Loss: 0.605862021446228
Loss: 0.1947467029094696
Loss: 0.5922020673751831
Epoch Loss: 0.468313440508435
Saving...
--- Epoch [4/5] ---
Loss: 0.5997691750526428
Loss: 0.6881994605064392
Loss: 0.5229768753051758
Loss: 0.6171546578407288
Loss: 0.24648571014404297
Epoch Loss: 0.4588907350706449
Saving...
--- Epoch [5/5] ---
Loss: 0.4747392535209656
Loss: 0.5189821720123291
Loss: 0.42935848236083984
Loss: 0.5588717460632324
Loss: 0.351311057806015
Epoch Loss: 0.4622343526357316
Saving...


In [10]:
# sampling loop
U = Unet(dim=128, channels=1, dim_mults=(1, 2, 4, 8), self_condition=True)
if torch.cuda.is_available():
    U.cuda()
U.load_state_dict(torch.load('cond_diffusion.pth'))

U.eval()
total_loss = 0.
total_elems = 0
for step, batch in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
    cond, targets = batch
    cond = torch.from_numpy(scaler.transform(cond))
    cond, targets = cond.float(), targets.float()
    cond, targets = cond.to(device), targets.to(device)
    targets = targets.unsqueeze(0).unsqueeze(0)
    cond = cond.unsqueeze(0)
    targets = torch.nn.functional.pad(targets, (0, 71, 0, 9), "constant", 0)
    # torch inference_mode already annotated for sample() function
    samples = sample(U, (64, 256), batch_size=1, channels=1, time_cond=cond)
    total_loss += torch.sum(torch.abs(torch.from_numpy(samples[-1]) - targets.detach().cpu())).item()
    total_elems += torch.numel(targets)
    print(total_loss / total_elems)
print(f"MAE: {total_loss / total_elems}")

  2%|▏         | 1/48 [01:18<1:01:52, 78.99s/it]

4.113003730773926


  4%|▍         | 2/48 [02:37<1:00:33, 78.98s/it]

4.315007448196411


  6%|▋         | 3/48 [03:56<59:13, 78.97s/it]  

...

4.303508618603582


 98%|█████████▊| 47/48 [1:01:50<01:18, 78.92s/it]

4.297000174826764


100%|██████████| 48/48 [1:03:09<00:00, 78.94s/it]

4.28546083966891
MAE: 4.28546083966891


# TimeGrad-Adapted Conditional Diffusion with FRFT

Here we perform our modified TimeGrad and add on conditioning on the Fractional Fourier transform.

In [8]:
from fft_cond_diffusion_model import Unet, p_losses, sample
U = Unet(dim=128, channels=1, dim_mults=(1, 2, 4, 8), self_condition=True)
if torch.cuda.is_available():
    U.cuda()
    print('Models moved to GPU.')
u_optimizer = torch.optim.Adam(U.parameters(), 0.0001, [0.5, 0.999])

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Models moved to GPU.


In [9]:
num_epc = 5
for epoch in range(num_epc):
    print(f"--- Epoch [{epoch+1}/{num_epc}] ---")

    total_epoch_loss = 0.

    for step, batch in enumerate(train_dataloader):
        cond, targets = batch
        cond = torch.from_numpy(scaler.transform(cond))
        cond, targets = cond.float(), targets.float()
        cond, targets = cond.to(device), targets.to(device)
        targets = targets.unsqueeze(0).unsqueeze(0)
        cond = cond.unsqueeze(0)
        targets = torch.nn.functional.pad(targets, (0, 71, 0, 9), "constant", 0)

        # 1. Sample t uniformally for every example in the batch
        t = torch.randint(low=0, high=500, size=(1,), device=device).long()

        # 2. Get l1 loss
        loss = p_losses(U, targets, t, loss_type='l2', time_cond=cond)

        if step % 100 == 0:
            print("Loss:", loss.item())
    
        total_epoch_loss += loss.detach().item()

        u_optimizer.zero_grad()
        loss.backward()
        u_optimizer.step()
    
    print("Epoch Loss:", total_epoch_loss / len(train_dataloader))

    # Save every epoch
    print("Saving...")
    torch.save(U.state_dict(), "frft_cond_diffusion.pth")

--- Epoch [1/5] ---
Loss: 1.3692975044250488
Loss: 0.22155994176864624
Loss: 0.606671929359436
Loss: 0.29038989543914795
Loss: 0.7157468795776367
Epoch Loss: 0.5967875802351751
Saving...
--- Epoch [2/5] ---
Loss: 0.48445069789886475
Loss: 0.624477207660675
Loss: 0.2906903326511383
Loss: 0.6239463686943054
Loss: 0.19061361253261566
Epoch Loss: 0.48953074664221624
Saving...
--- Epoch [3/5] ---
Loss: 0.16034150123596191
Loss: 0.12739458680152893
Loss: 0.6061078906059265
Loss: 0.1953548789024353
Loss: 0.5962360501289368
Epoch Loss: 0.4697800937687277
Saving...
--- Epoch [4/5] ---
Loss: 0.6069482564926147
Loss: 0.6854190826416016
Loss: 0.5220898389816284
Loss: 0.6204911470413208
Loss: 0.24741126596927643
Epoch Loss: 0.45765033223199514
Saving...
--- Epoch [5/5] ---
Loss: 0.47280484437942505
Loss: 0.515881359577179
Loss: 0.42626556754112244
Loss: 0.5580533742904663
Loss: 0.3658735454082489
Epoch Loss: 0.46285366972448094
Saving...


In [11]:
# sampling loop
U = Unet(dim=128, channels=1, dim_mults=(1, 2, 4, 8), self_condition=True)
if torch.cuda.is_available():
    U.cuda()
U.load_state_dict(torch.load('frft_cond_diffusion.pth'))

U.eval()
total_loss = 0.
total_elems = 0
for step, batch in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
    cond, targets = batch
    cond = torch.from_numpy(scaler.transform(cond))
    cond, targets = cond.float(), targets.float()
    cond, targets = cond.to(device), targets.to(device)
    targets = targets.unsqueeze(0).unsqueeze(0)
    cond = cond.unsqueeze(0)
    targets = torch.nn.functional.pad(targets, (0, 71, 0, 9), "constant", 0)
    # torch inference_mode already annotated for sample() function
    samples = sample(U, (64, 256), batch_size=1, channels=1, time_cond=cond)
    total_loss += torch.sum(torch.abs(torch.from_numpy(samples[-1]) - targets.detach().cpu())).item()
    total_elems += torch.numel(targets)
    print(total_loss / total_elems)
print(f"MAE: {total_loss / total_elems}")

  2%|▏         | 1/48 [01:22<1:04:20, 82.13s/it]

4.11014461517334


  4%|▍         | 2/48 [02:44<1:02:55, 82.07s/it]

4.281827926635742


  6%|▋         | 3/48 [04:06<1:01:32, 82.06s/it]

...

4.254121687101281


 98%|█████████▊| 47/48 [1:04:09<01:21, 81.82s/it]

4.2482460356773215


100%|██████████| 48/48 [1:05:31<00:00, 81.90s/it]

4.234675755103429
MAE: 4.234675755103429
